In [4]:
import pandas as pd
import numpy as np
from glob import glob
import os

In [24]:
# ======================================================
# 1. Load Karachi MONTHLY data (you already generated)
# ======================================================

karachi_file = "karachi_monthly_pm25.csv"  
karachi = pd.read_csv(karachi_file)

# Standardize
karachi["date"] = pd.to_datetime(karachi["YearMonth"])
karachi["pm25"] = karachi["Raw Conc."]
karachi["country"] = "Pakistan"
karachi["city"] = "Karachi"
karachi = karachi[["country","city","date","pm25"]]

print("Loaded Karachi monthly data:", karachi.shape)

Loaded Karachi monthly data: (68, 4)


In [ ]:
# ======================================================
# 1. Load Karachi MONTHLY data (you already generated)
# ======================================================

karachi_file = "karachi_dhaphas_monthly_pm25.csv"  
karachi = pd.read_csv(karachi_file)

# Standardize
karachi["date"] = pd.to_datetime(karachi["date"])
karachi["country"] = "Pakistan"
karachi["city"] = "Karachi"
karachi = karachi[["country","city","date","pm25"]]

print("Loaded Karachi monthly data:", karachi.shape)

In [21]:
# ======================================================
# 2. Load ALL donor files (hourly PM2.5)
# ======================================================

donor_folder = "pm25_donors/*.csv"  # <-- CHANGE PATH
files = glob(donor_folder)

print("Found donor files:", len(files))

all_monthly_donors = []

# Mapping from city names found in filenames → country
city_to_country = {
    "Dhaka": "Bangladesh",
    "Dushanbe": "Tajikistan",
    "Rangoon": "Myanmar",
    #"Baghdad": "Iraq",
    #"Dhahran": "SaudiArabia"
    #"Jeddah": "SaudiArabia"
}

for f in files:
    print(f"Processing:", f)

    df = pd.read_csv(f)

    # --- Standardize column names (but keep original capitalization for safety) ---
    df.columns = df.columns.str.strip()

    # Check required columns exist
    if "Date (LT)" not in df.columns or "Raw Conc." not in df.columns:
        raise ValueError(f"Unexpected donor file format: {f}")

    # --- Parse datetime ---
    df["date"] = pd.to_datetime(df["Date (LT)"], errors="coerce")
    df = df.dropna(subset=["date"])

    # --- Extract PM2.5 ---
    df["pm25"] = pd.to_numeric(df["Raw Conc."], errors="coerce")
    # Clean invalid PM2.5 values
    df.loc[df["pm25"] < 0, "pm25"] = np.nan      # remove -999 etc.
    df.loc[df["pm25"] > 1000, "pm25"] = np.nan   # remove sensor overflow
    df = df.dropna(subset=["pm25"])

    # --- Optional: keep only valid QC ---
    # df = df[df["QC Name"] == "Valid"]

    # --- Identify city from filename ---
    filename = os.path.basename(f)          # Works on Windows/macOS/Linux
    name_no_ext = filename.replace(".csv", "")

    # Example filenames:
    #   Baghdad_PM2.5_2019_YTD.csv
    #   Dhaka_PM2.5_2017.csv
    # So split by "_" and take the first piece:
    city_guess = name_no_ext.split("_")[0]

    if city_guess not in city_to_country:
        raise ValueError(f"City not recognized: '{city_guess}' extracted from '{filename}'")

    country = city_to_country[city_guess]

    # --- Compute MONTHLY average ---
    df["year_month"] = df["date"].dt.to_period("M")
    monthly = df.groupby("year_month")["pm25"].mean().reset_index()
    monthly["date"] = monthly["year_month"].dt.to_timestamp()

    # Add city & country
    monthly["city"] = city_guess
    monthly["country"] = country

    monthly = monthly[["country","city","date","pm25"]]

    all_monthly_donors.append(monthly)


Found donor files: 18
Processing: pm25_donors\Dhaka_PM2.5_2016_YTD.csv
Processing: pm25_donors\Dhaka_PM2.5_2017_YTD.csv
Processing: pm25_donors\Dhaka_PM2.5_2018_YTD.csv
Processing: pm25_donors\Dhaka_PM2.5_2019_YTD.csv
Processing: pm25_donors\Dhaka_PM2.5_2020_YTD.csv
Processing: pm25_donors\Dhaka_PM2.5_2021_YTD.csv
Processing: pm25_donors\Dhaka_PM2.5_2022_YTD.csv
Processing: pm25_donors\Dhaka_PM2.5_2023_YTD.csv
Processing: pm25_donors\Dushanbe_PM2.5_2019_YTD.csv


C:\Users\Jinho Kim\AppData\Local\Temp\ipykernel_29108\832173828.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["Date (LT)"], errors="coerce")


Processing: pm25_donors\Dushanbe_PM2.5_2020_YTD.csv
Processing: pm25_donors\Dushanbe_PM2.5_2021_YTD.csv
Processing: pm25_donors\Dushanbe_PM2.5_2022_YTD.csv
Processing: pm25_donors\Dushanbe_PM2.5_2023_YTD.csv
Processing: pm25_donors\Rangoon_PM2.5_2019_YTD.csv


C:\Users\Jinho Kim\AppData\Local\Temp\ipykernel_29108\832173828.py:35: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df["date"] = pd.to_datetime(df["Date (LT)"], errors="coerce")


Processing: pm25_donors\Rangoon_PM2.5_2020_YTD.csv
Processing: pm25_donors\Rangoon_PM2.5_2021_YTD.csv
Processing: pm25_donors\Rangoon_PM2.5_2022_YTD.csv
Processing: pm25_donors\Rangoon_PM2.5_2023_YTD.csv


In [22]:
# ======================================================
# 3. Combine ALL donors
# ======================================================

donors = pd.concat(all_monthly_donors, ignore_index=True)
print("Donor monthly dataset shape:", donors.shape)

Donor monthly dataset shape: (223, 4)


In [25]:
# ======================================================
# 4. Combine Karachi + Donors → FINAL PANEL
# ======================================================

panel = pd.concat([karachi, donors], ignore_index=True)

panel = panel.sort_values(["country","city","date"])
panel.to_csv("pm25_panel.csv", index=False)

print("\nSaved merged dataset to pm25_panel.csv")
print(panel.head())
print(panel.tail())


Saved merged dataset to pm25_panel.csv
       country   city       date        pm25
68  Bangladesh  Dhaka 2016-03-01  108.788410
69  Bangladesh  Dhaka 2016-04-01   49.787204
70  Bangladesh  Dhaka 2016-05-01   51.318548
71  Bangladesh  Dhaka 2016-06-01   39.015406
72  Bangladesh  Dhaka 2016-07-01   29.364738
        country      city       date       pm25
225  Tajikistan  Dushanbe 2023-09-01  63.004918
226  Tajikistan  Dushanbe 2023-10-01  43.313669
227  Tajikistan  Dushanbe 2023-11-01  46.177778
228  Tajikistan  Dushanbe 2023-12-01  79.358008
229  Tajikistan  Dushanbe 2024-01-01  36.000000
